In [ ]:
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive
!pip install vecstack

In [ ]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 

In [ ]:
import warnings
warnings.filterwarnings("ignore")
trainfile = r'/gdrive/My Drive/CIS508/NewTrain.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/CIS508/NewTest.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 

print(test_data.shape)
print(test_data.head()) 

In [ ]:
X_train = train_data.drop(["QuoteConversion_Flag"],axis=1)
X_test = test_data
print(X_train)
print(X_test)
combined_Data = pd.concat([train_data, test_data], keys=[0,1])

In [ ]:
percent=200000
print(percent)
td=combined_Data.dropna(thresh=percent,axis=1)
print(td.shape)

numeric=td.select_dtypes(include=['int','float64']).columns
for num in numeric:
  td[num]=td[num].fillna(td[num].mean())

train_cat_cols = td.select_dtypes(exclude=['int','float64']).columns
for colss in train_cat_cols:
  if(td.iloc[0][colss]=="N"):
        td[colss]=td[colss].fillna("N")
  else:
    td[colss]=td[colss].fillna(td[colss].mode())


In [ ]:
print(td.head())
pd.options.display.max_columns = None
pd.options.display.max_rows = None
print (td.isnull().sum())
pd.options.display.max_columns = 5
pd.options.display.max_rows = 5
trainData = td.xs(0)
testData = td.xs(1)
print(trainData.shape)
print(testData.shape)
train_cat_cols = trainData.select_dtypes(exclude=['float','int']).columns 
print(train_cat_cols.shape)
print(train_cat_cols)
X_train = trainData
X_test = testData
y_train=train_data["QuoteConversion_Flag"]
print(X_train.head())
print(X_test.head()) 
print(y_train.head())
QuoteNumber = test_data.iloc[:,252]
print(QuoteNumber.head)

In [ ]:
# Normal Decision Tree

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
pred=pd.DataFrame(clf.predict(X_test),columns=["QuoteConversion_Flag"])

pred = pd.concat([QuoteNumber, pred],axis=(1))
pred.head()
pred.to_csv('/gdrive/My Drive/CIS508/DecisionTreeOriginal.csv',index= None)
#Decision Tree Hypertuning
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,10,2),"min_samples_leaf":range(10,100,10)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=5)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)
clf1 = DecisionTreeClassifier(**grid_parm)
clf1.fit(X_train,y_train)
clf1_predict = clf1.predict(X_test)

pred2=pd.DataFrame(clf1.predict(X_test),columns=["QuoteConversion_Flag"])
print(pred2.head())
pred2 = pd.concat([QuoteNumber, pred2],axis=(1))
pred2.head()
pred2.to_csv('/gdrive/My Drive/CIS508/DecisionTreeHypertuned.csv',index= None)

In [ ]:
#Random Forest Classifier
RFC= RandomForestClassifier(**grid_parm)
RFC.fit(X_train,y_train)
RFC_predict = RFC.predict(X_test)

pred3=pd.DataFrame(RFC.predict(X_test),columns=["QuoteConversion_Flag"])
pred_3 = pd.concat([QuoteNumber, pred3],axis=(1))
pred_3.head()
pred3.to_csv('/gdrive/My Drive/CIS508/DecisionTreeOriginal.csv',index= None)


In [ ]:
#SMOTE
print("SMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio="minority",m_neighbors="10",random_state=43)
X_rs, y_rs = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_rs))
print(y_train.value_counts()/len(y_train))
print(pd.Series(y_rs).value_counts()/len(y_rs))
#Decision Tree SMOTE
clf2 = DecisionTreeClassifier()
clf2.fit(X_rs, y_rs)
predA=pd.DataFrame(clf2.predict(X_test),columns=["QuoteConversion_Flag"])
predA = pd.concat([QuoteNumber, predA],axis=(1))
predA.head()
predA.to_csv('/gdrive/My Drive/CIS508/SMOTE-DecisionTree.csv',index= None)

In [ ]:
#KNN

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

knn=KNeighborsClassifier(5)
knn.fit(X_rs,y_rs)
knn_predict = knn.predict(X_test)

pred4=pd.DataFrame(knn.predict(X_test),columns=["QuoteConversion_Flag"])
predb = pd.concat([QuoteNumber, pred4],axis=(1))
predb.head()
predb.to_csv('/gdrive/My Drive/CIS508/KNN.csv',index= None)


In [ ]:
#SVC

from sklearn.svm import LinearSVC
SVC = LinearSVC()
SVC.fit(X_rs,y_rs)
SVC_predict = SVC.predict(X_test)

pred6=pd.DataFrame(SVC.predict(X_test),columns=["QuoteConversion_Flag"])
predd = pd.concat([QuoteNumber, pred6],axis=(1))
predd.head()
predd.to_csv('/gdrive/My Drive/CIS508/SVC.csv',index= None)

In [ ]:
#SMOTE- Random Forest

RFC1= RandomForestClassifier(**grid_parm)
RFC1.fit(X_rs,y_rs)
RFC1_predict = RFC1.predict(X_test)

pred5=pd.DataFrame(RFC.predict(X_test),columns=["QuoteConversion_Flag"])
predc = pd.concat([QuoteNumber, pred5],axis=(1))
predc.head()
predc.to_csv('/gdrive/My Drive/CIS508/SMOTE-RandomForest.csv',index= None)


In [ ]:
#Stacking

print("Ensemble Methods Predictions using RandomForest, Decision Tree Classifier, Linear SVC\n")

models = [ RandomForestClassifier(**grid_parm), DecisionTreeClassifier(), LinearSVC()]
      
S_Train, S_Test = stacking(models,                   
                           X_rs, y_rs, X_test,   
                           regression=False, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
            
                           random_state=0,    
         
                           verbose=2)
model2= GradientBoostingClassifier()
parameters={'min_samples_split' : range(20,100,20),'max_depth': range(1,10,5),"min_samples_leaf":range(20,100,20)}
model2_random = RandomizedSearchCV(model,parameters,n_iter=5)
model2_random.fit(S_Train, y_rs)
grid_parm=model_random.best_params_
print(grid_parm)
mode13 = GradientBoostingClassifier(**grid_parm)
model3.fit(S_Train, y_rs)
model3_predict = model3.predict(S_Test)
pred8=pd.DataFrame(model1.predict(S_Test),columns=["QuoteConversion_Flag"])
predf = pd.concat([QuoteNumber, pred8],axis=(1))
predfhead()
predf.to_csv('/gdrive/My Drive/CIS508/Stacked.csv',index= None)